Question 3  
3.1 Using the provided dataset, preprocess the data with CountVectorizer and train a Naive Bayes classifier to predict whether a given command should be classified as "Allow" or "Deny". 

In [37]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB # MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score #classification_report

In [20]:
data = {
    "spoken_phrase": ['grant access', 'deny access', 'approve access', 'block access', 'let in'],
    "action": ['Allow', 'Deny', 'Allow', 'Deny', 'Allow']
}

In [38]:
df = pd.DataFrame(data)

# selecting features and label
X = df['spoken_phrase']
y = df['action']

# applying countervectoriser
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(X).toarray()

"""# tf-idf vectorizer
tf = TfidfVectorizer()
X_vec = tf.fit_transform(X).toarray()"""

print(f"X vectorized shape: {X_vec.shape}")


X vectorized shape: (5, 7)


In [39]:
# creating model 
# gaussian model
gnb = GaussianNB()
gnb.fit(X_vec, y)

gnb_y_pred = gnb.predict(X_vec)
print(f"GaussianNB predictions: {gnb_y_pred}")

accuracy = accuracy_score(y, gnb_y_pred)
print(f"GaussianNB accuracy: {accuracy*100:.2f}%")

# multinomial model
#mnb = MultinomialNB()
#mnb.fit(X_vec, y)
#mnb_y_pred = mnb.predict(X_vec)
#print(f"MultinomialNB predictions: {mnb_y_pred}")


GaussianNB predictions: ['Allow' 'Deny' 'Allow' 'Deny' 'Allow']
GaussianNB accuracy: 100.00%


3.2 Predict the intent of a new command: "block entry"

Also, calculate and display the probability scores for each class (Allow/Deny). 

In [40]:
# testing the model
def predict_command(command):
    phrase_vec = tf.transform([command]).toarray()
    pred = gnb.predict(phrase_vec)[0]
    prob = gnb.predict_proba(phrase_vec)[0]
    return pred, prob

# showing the system is ready
print('System is ready for testing!')

System is ready for testing!


In [41]:
# test input data
command = 'block entry'
result, probs = predict_command(command)

print(f"Test phrase: '{command}'\n Prediction: {result}\n Probabilities: {probs}")

Test phrase: 'block entry'
 Prediction: Deny
 Probabilities: [1.1993559e-14 1.0000000e+00]


3.3 Write a Python function that publishes the result to a topic named /gate_command. 

In [45]:
def gate_command(command, confidence):
    # emulating the publishing of the gate command
    msgData = f"{command},{confidence:.3f}"
    print(f"Emulated publish to /gate_command: {msgData}", flush=True)

In [8]:
def voice_to_gate(spoken_phrase):
    # predicting intent using the model
    pred, prob = predict_command(spoken_phrase)
    confidence = max(prob)
    # publish command to gate control system
    gate_command(pred, confidence)

    # logging interactions
    print(f"Voice command: '{spoken_phrase}")
    print(f"Prediction: {pred} (confidence: {confidence:.3f})")
    print(f"Sent to gate: /gate_command -> {pred}:{confidence:.3f}")

    return pred, confidence

In [47]:
# Test the gate_command function directly
gate_command("open_gate", 0.987)

Emulated publish to /gate_command: open_gate,0.987


In [48]:
voice_to_gate("open the date")

Emulated publish to /gate_command: Allow,1.000
Voice command: 'open the date
Prediction: Allow (confidence: 1.000)
Sent to gate: /gate_command -> Allow:1.000


('Allow', 1.0)

In [49]:
voice_to_gate("please deny access")

Emulated publish to /gate_command: Deny,1.000
Voice command: 'please deny access
Prediction: Deny (confidence: 1.000)
Sent to gate: /gate_command -> Deny:1.000


('Deny', 1.0)